# 1 The objective of this notebook is to analyze the data from the dataset and set up a reference metric with a baseline model

## Data description

Data Description:

The data comes from direct marketing efforts of a European banking institution. The marketing campaign involves making a phone call to a customer, often multiple times to ensure a product subscription, in this case a term deposit. Term deposits are usually short-term deposits with maturities ranging from one month to a few years. The customer must understand when buying a term deposit that they can withdraw their funds only after the term ends. All customer information that might reveal personal information is removed due to privacy concerns.

Attributes:

age : age of customer (numeric)

job : type of job (categorical)

marital : marital status (categorical)

education (categorical)

default: has credit in default? (binary)

balance: average yearly balance, in euros (numeric)

housing: has a housing loan? (binary)

loan: has personal loan? (binary)

contact: contact communication type (categorical)

day: last contact day of the month (numeric)

month: last contact month of year (categorical)

duration: last contact duration, in seconds (numeric)

campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

Output (desired target):

y - has the client subscribed to a term deposit? (binary)

In [2]:
#---------Importing libraries---------#

#---Data analysis---#
import pandas as pd
import numpy as np


#---Data visualization---#
import matplotlib.pyplot as plt
import seaborn as sns

#---Data preprocessing---#
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC

#---Modeling---#
from sklearn.linear_model import LogisticRegression

#---utils---#
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


## Step 1: Data Loading and Inspection

 - Load the dataset into a pandas DataFrame.
- Check the dimensions of the dataset (number of rows and columns).
- Examine the first few rows of the dataset to get an initial understanding of the data.
- Check for any missing values in the dataset.

In [3]:
df = pd.read_csv('../data/raw/term-deposit-marketing-2020.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/term-deposit-marketing-2020.csv'

In [ ]:
df.info()

: 

We can see that  the dataset have none missing values, and a lot of datapoints, 40k.
Also, There is features as objects, that could be all into the category data type.
We will first change the data type of the features that are objects, and then we will see the distribution of the data.
For this, we want to know the number of unique features in each column to decide how we are going to categorize the object columns.

## Step 2: Data Cleaning and Preprocessing

 - Handle missing values, if any, by either imputing them or removing rows/columns with missing values, depending on the situation.
- Convert categorical variables into appropriate data types (e.g., convert 'month' and 'contact' to categorical data type).
- Encode binary categorical variables ('default', 'housing', 'loan', and 'y') into numeric format (0 and 1) for modeling purposes.

In [ ]:
#checking unique values in each column when the column type is object


for col in df.columns:
    if df[col].dtype == 'object':
        print(col, df[col].nunique())

: 

In [ ]:
#converting month into datetime, when the data is the name of the month only
df['month'] = pd.to_datetime(df['month'], format='%b').dt.month

#results
df['month'].dtypes

: 

In [ ]:
df['month'].head()

: 

In [ ]:
#storing binary and then categorical data
#binary data
binary_features = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtypes == 'object' and col != 'y']
# categorical data
categorical_features = [col for col in df.columns if  df[col].nunique() > 2 and df[col].dtypes == 'object']

#showing results
print('Binary features: ', binary_features)
print('Categorical features: ', categorical_features)

: 

In [ ]:
#values of the columns
print('Binary features\n')
for col in binary_features:
    print('Column_binary: ', col)
    print('Unique values: ', df[col].unique())
    print('Number of unique values: ', len(df[col].unique()))

print('\nCategorical  features\n')
for col in categorical_features:
    print('Column_categorical: ', col)
    print('Unique values: ', df[col].unique())
    print('Number of unique values: ', len(df[col].unique()))
    print('\n')

: 

In [ ]:
#converting data to the right dtype. Also, age will be converted to a categorical variable

# defing the funtion to convert the data to the right dtype
def convert_features(df: object):
    """
    Converts the data to the right dtype
    """
    binary_features = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtypes == 'object' ]
    categorical_features = [col for col in df.columns if df[col].nunique() > 2 and df[col].dtypes == 'object' and col != 'y']
    

    # converting binary features to 0 and 1
    df[binary_features] = df[binary_features].replace({'no': 0, 'yes': 1})

    # converting categorical features to dummy variables
    df[categorical_features] = df[categorical_features].astype('category')


   
    

    return df
    


: 

In [ ]:
#converting the data into a dataframe with desired dtypes

convert_features(df)

df.info()

: 

In [ ]:
#UPdate categorical columns
categorical_features = categorical_features + ['age']
categorical_features

: 

In [ ]:
#well, lets check the unique valuesof the categories that has unkown values.

print("Unique values of the categories that has unkown values")
print('contact:', df[df['contact'] == 'unknown'].shape[0])
print('Percentage of unknown values in contact:', round(df[df['contact'] == 'unknown'].shape[0]/df.shape[0]*100, 2), '%')

print('education:', df[df['education'] == 'unknown'].shape[0])
print('Percentage of unknown values in education:', round(df[df['education'] == 'unknown'].shape[0]/df.shape[0]*100, 2), '%')

: 

Later we could check the importance of these two features to see if they matter for the model. If they matter, we can drop the rows with missing values. If they don't matter, we can drop the columns.

As rule of thumb, we could drop the rows of the dataset if the percentage of missing values is low liek 'education' (say, less than 5%). Otherwise, we should drop the columns, unless the part of it that is not missing is very important for the model.

In [ ]:
df.head()

: 

## Step 3: Data Exploration and Visualization

- Explore the distribution of the target variable 'y' (term deposit subscription) to understand the class balance (whether it's imbalanced or not).
- Analyze the distribution of each numerical feature (e.g., 'age', 'balance', 'day', 'duration', 'campaign') using histograms and other visualizations.
- Visualize the relationships between the target variable and other numerical features using box plots, scatter plots, or violin plots, as well as correlation matrix plots.
- Examine the distribution of categorical features ('job', 'marital', 'education', 'month') using bar plots or count plots.



In [ ]:
# checking for data target y imbalance

sns.countplot(x='y', data=df)
plt.title('Target y distribution')
plt.show()

: 

In [ ]:
#native pandas way to check the data values
df.y.value_counts().plot(kind='bar', title='Number of samples per class');

: 

This data is very unbalanced, and we will need to take this into account when we train our model. We will use the SMOTE algorithm to oversample the minority class. This will help us to avoid overfitting and improve the accuracy of our model.

## Step 4: Feature Engineering

- Create new features if necessary. For example, you could create a binary feature indicating whether a customer was contacted on a weekday or weekend based on the 'day' column, as well as categoryzing the age feature

- Explore feature interactions and transformations that might be useful for modeling.

In [ ]:
# checking for age range
df['age'].describe()

: 

In [ ]:
# converting age to categorical variable
df['age'] = pd.cut(df['age'], bins=[18, 30, 60, 100], labels=['adult', 'mid-aged', 'elderly'])


: 

In [ ]:
#checnking our head of the df
df.head()

: 

In [ ]:
#we now check our categorical data

for cat in categorical_features:
    print(f'\n{cat} has {df[cat].unique()}')

: 

We could add more features with dummy variables for the categorical features, but we will just keep the same features and transform them into numerical values with categorical codes.

In [ ]:
#getting our codes for our categorical data
#creating an instance of the LabelEncoder
le = LabelEncoder()

#fitting the LabelEncoder to our categorical data
encoded_labels = le.fit_transform(df[categorical_features])


#creating a dataframe of the encoded labels
encoded_df = pd.DataFrame(encoded_labels, columns=categorical_features)

#checking the head of the encoded dataframe
encoded_df.head()







: 

In [ ]:
# getting our dataset into the feature matrix and the target vector

X = df.drop('y', axis=1)
y = df['y']

print('X shape: ', X.shape)
print('y shape: ', y.shape)



: 

In [ ]:
#Dealing with data imbalance with SMOTE to oversample the minority class
#https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

smote = SMOTENC(random_state=42, categorical_features=categorical_features, sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

sns.countplot(y_sm)
plt.title('Balanced Class Distributions', fontsize=14);


: 

Nice, now our dataset is balanced with SMOTENC, a oversampling techinique for categorical and continuous features. It uses k-nearest neighbors to create synthetic samples using the distances of the classes clusters, so it create new data points between the existing ones. This is a good technique to use when we have a small dataset and we want to balance it.

In [ ]:
#we then finally create our training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

: 

## Step 5: Baseline Model

- Select a simple machine learning model as a baseline, such as Logistic Regression or Decision Tree Classifier.
- Split the data into training and testing sets.
- Train the baseline model using the training data.
- Evaluate the model's performance on the testing data using appropriate evaluation metrics (e.g., accuracy, precision, recall, F1-score, ROC-AUC).

In [ ]:
#creating a baseline model and cheking for the accuracy

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('Accuracy: ', lr.score(X_test, y_test))


: 

## Step 6: Interpretation and Analysis

- Interpret the baseline model's results and identify areas of improvement.
- Analyze which features have the most significant impact on the target variable.

## Step 7: Future Steps and Iteration

- Based on the insights gained from EDA and the baseline model, plan the next steps in the data science project, which may include more advanced modeling techniques, feature selection, hyperparameter tuning, and model evaluation.
